In [4]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
train_df = pd.read_csv(r"dataset\fashion-mnist_train.csv")
test_df = pd.read_csv(r"dataset\fashion-mnist_test.csv")

train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_data = np.array(train_df, dtype='float32')
test_data = np.array(test_df,dtype='float32')

x_train = train_data[:,1:]/255
y_train = train_data[:,0]

x_test = test_data[:,1:]/255
y_test = test_data[:,0]

In [7]:
x_train,x_validate,y_train,y_validate = train_test_split(
    x_train, y_train, test_size=0.2, random_state=12345,
)

In [8]:
im_rows = 28
im_cols = 28
batch_size = 512
im_shape = (im_rows,im_cols,1)

x_train = x_train.reshape(x_train.shape[0],*im_shape)
x_test = x_test.reshape(x_test.shape[0],*im_shape)
x_validate = x_validate.reshape(x_validate.shape[0],*im_shape)

print('x_train shape : {}'.format(x_train.shape))
print('x_test shape : {}'.format(x_test.shape))
print('x_validate shape : {}'.format(x_validate.shape))

x_train shape : (48000, 28, 28, 1)
x_test shape : (10000, 28, 28, 1)
x_validate shape : (12000, 28, 28, 1)


In [9]:
cnn_model = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', 
           input_shape=im_shape, kernel_initializer='he_normal', name='Conv2D-1'),
    MaxPooling2D(pool_size=2, name='MaxPool'),
    Dropout(0.25, name='Dropout-1'),
    Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2'),
    Dropout(0.25, name='Dropout-2'),
    Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3'),
    Dropout(0.4, name='Dropout-3'),
    Conv2D(256, kernel_size=3, activation='relu', name='Conv2D-4'),
    Dropout(0.6, name='Dropout-4'),
    Flatten(name='flatten'),
    Dense(256, activation='relu', name='Dense'),
    Dropout(0.6, name='Dropout'),
    Dense(10, activation='softmax', name='Output')
])

In [10]:
cnn_model.compile(
    loss ='sparse_categorical_crossentropy',
    optimizer = Adam(lr=0.001),
    metrics=['accuracy']
)

In [11]:
cnn_model.fit(
    x_train, y_train, batch_size=batch_size,
    epochs=50, verbose=1,
    validation_data=(x_validate,y_validate),
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 38s 792us/step - loss: 0.9456 - acc: 0.6533 - val_loss: 0.5143 - val_acc: 0.7971
Epoch 2/50
48000/48000 [==============================] - 10s 206us/step - loss: 0.5387 - acc: 0.8009 - val_loss: 0.4011 - val_acc: 0.8533
Epoch 3/50
48000/48000 [==============================] - 10s 207us/step - loss: 0.4467 - acc: 0.8374 - val_loss: 0.3408 - val_acc: 0.8750
Epoch 4/50
48000/48000 [==============================] - 10s 206us/step - loss: 0.3970 - acc: 0.8582 - val_loss: 0.2944 - val_acc: 0.8920
Epoch 5/50
48000/48000 [==============================] - 10s 207us/step - loss: 0.3712 - acc: 0.8676 - val_loss: 0.2907 - val_acc: 0.8919
Epoch 6/50
48000/48000 [==============================] - 10s 208us/step - loss: 0.3473 - acc: 0.8754 - val_loss: 0.2635 - val_acc: 0.9015
Epoch 7/50
48000/48000 [==============================] - 10s 207us/step - loss: 0.3277 - acc: 0.8821 - val_loss: 0.

In [12]:
score = cnn_model.evaluate(x_test,y_test,verbose=0)

print('test loss : {:4f}'.format(score[0]))
print('test score : {:4f}'.format(score[1]))

test loss : 0.175191
test score : 0.937500
